In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### 1. 데이터 불러오기

In [ ]:
path = "/content/drive/MyDrive/team_project1/data/"

In [ ]:
df = pd.read_csv(path + "Regression_data_preprocessing.csv")
print(df.shape)
df.head()

(4177, 11)


,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


### 2. 데이터 나누기
- train : test = 0.8 : 0.2
- train : test = 0.7 : 0.3

In [ ]:
X = df.drop('Rings', axis=1)
y = df['Rings']

X.shape, y.shape

((4177, 10), (4177,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((3341, 10), (836, 10))

### 3. 학습 관련 함수 정의
- 손실 계산 함수 -> 베이스 모델 함수와 동일한 지표
- 정확도 계산 함수 -> 베이스 모델 함수와 동일한 지표
- 예측한 결과와 원래 결과를 비교해서 시각화

In [ ]:
# loss
def mse(y_true, y_pred):
  return mean_squared_error(y_true, y_pred)

In [ ]:
# 정확도 계산
def eval_accuracy(y, y_hat):
	# 오차율 구하는 과정    
	# np.mean() 메서드의 이유는 미니배치 처리를 고려하여 하나의 지표로 묶어주기 위함 입니다. 
    mdiff = np.mean(np.abs((y_hat - y) / y))
    # 1 에서 오차율을 빼 정확도를 구합니다. 
    return 1 - mdiff

In [ ]:
# test_count = y_test.value_counts().sort_values(ascending=False)
# test_count.plot(kind='bar')

### 4. 기본 선형 모델

In [ ]:
# 모델 정의
lr = LinearRegression()

In [ ]:
# 학습
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
# 예측
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [ ]:
print(f"Train - Loss = {mse(y_train, y_train_pred):.3f}, Accuracy = {eval_accuracy(y_train, y_train_pred):.3f} / Test - Accuracy = {mse(y_test, y_test_pred):.3f}, Accuracy = {eval_accuracy(y_test, y_test_pred):.3f}")

Train - Loss = 4.666, Accuracy = 0.843 / Test - Accuracy = 5.048, Accuracy = 0.841


### 5. 릿지 선형 회귀 모델

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# 모델 정의 
ridge_model = Ridge()

In [ ]:
# 하이퍼파라미터 값 목록
ridge_parmas = {
    'max_iter': [3000],
    'alpha': [0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000]
}

In [ ]:
gridsearch_ridge_model = GridSearchCV(estimator=ridge_model,
                                      param_grid=ridge_parmas,
                                      return_train_score=True,
                                      cv=5)

In [ ]:
gridsearch_ridge_model.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400,
                                   800, 900, 1000],
                         'max_iter': [3000]},
             return_train_score=True)

In [ ]:
print(f'최적 하이퍼파라미터 : {gridsearch_ridge_model.best_params_}')

최적 하이퍼파라미터 : {'alpha': 0.1, 'max_iter': 3000}


In [ ]:
# 예측
y_train_pred = gridsearch_ridge_model.best_estimator_.predict(X_train)
y_test_pred = gridsearch_ridge_model.best_estimator_.predict(X_test)

In [ ]:
print(f"Train - Loss = {mse(y_train, y_train_pred):.3f}, Accuracy = {eval_accuracy(y_train, y_train_pred):.3f} / Test - Accuracy = {mse(y_test, y_test_pred):.3f}, Accuracy = {eval_accuracy(y_test, y_test_pred):.3f}")

Train - Loss = 4.668, Accuracy = 0.843 / Test - Accuracy = 5.058, Accuracy = 0.841


### 6. 라쏘 회귀 모델

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_model = Lasso()

In [ ]:
# 하이퍼파라미터 값 목록
lasso_parmas = {
    'max_iter': [3000],
    'alpha': 1 / np.array([0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000])
}

In [ ]:
gridsearch_lasso_model = GridSearchCV(estimator=lasso_model,
                                      param_grid=lasso_parmas,
                                      return_train_score=True,
                                      cv=5)

In [ ]:
gridsearch_lasso_model.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([1.00000000e+01, 1.00000000e+00, 5.00000000e-01, 3.33333333e-01,
       2.50000000e-01, 1.00000000e-01, 3.33333333e-02, 1.00000000e-02,
       5.00000000e-03, 3.33333333e-03, 2.50000000e-03, 1.25000000e-03,
       1.11111111e-03, 1.00000000e-03]),
                         'max_iter': [3000]},
             return_train_score=True)

In [ ]:
print(f'최적 하이퍼파라미터 : {gridsearch_lasso_model.best_params_}')

최적 하이퍼파라미터 : {'alpha': 0.001, 'max_iter': 3000}


In [ ]:
# 예측
y_train_pred = gridsearch_ridge_model.best_estimator_.predict(X_train)
y_test_pred = gridsearch_ridge_model.best_estimator_.predict(X_test)

In [ ]:
print(f"Train - Loss = {mse(y_train, y_train_pred):.3f}, Accuracy = {eval_accuracy(y_train, y_train_pred):.3f} / Test - Accuracy = {mse(y_test, y_test_pred):.3f}, Accuracy = {eval_accuracy(y_test, y_test_pred):.3f}")

Train - Loss = 4.668, Accuracy = 0.843 / Test - Accuracy = 5.058, Accuracy = 0.841


### 7. 랜덤 포레스트 회귀 모델

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# 모델 생성
rf_model = RandomForestRegressor()

In [ ]:
# 그리드 서치
rf_parmas = {
    'random_state':[42],
    'n_estimators':[100, 120, 140]
}

gridsearch_rf_model = GridSearchCV(estimator=rf_model,
                                   param_grid=rf_parmas,
                                   cv=5)

In [ ]:
gridsearch_rf_model.fit(X_train, y_train)
print(f'최적 하이퍼파라미터 : {gridsearch_rf_model.best_params_}')

최적 하이퍼파라미터 : {'n_estimators': 140, 'random_state': 42}


In [ ]:
# 예측
y_train_pred = gridsearch_rf_model.best_estimator_.predict(X_train)
y_test_pred = gridsearch_rf_model.best_estimator_.predict(X_test)

print(f"Train - Loss = {mse(y_train, y_train_pred):.3f}, Accuracy = {eval_accuracy(y_train, y_train_pred):.3f} / Test - Loss = {mse(y_test, y_test_pred):.3f}, Accuracy = {eval_accuracy(y_test, y_test_pred):.3f}")

Train - Loss = 0.642, Accuracy = 0.945 / Test - Loss = 4.920, Accuracy = 0.846
